# Train-val-test-split


In [ ]:
# import splitfolders
# input_folder = "C:/Users/Лавандос/Downloads/Telegram Desktop/Augmented"
# output = "C:/Users/Лавандос/Downloads/Telegram Desktop/AugmentedDataSet"
# splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.7, .15, .15))

Copying files: 15604 files [00:17, 881.20 files/s] 


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!7z x "/content/gdrive/MyDrive/birdsFinal.7z" -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/                                  1 file, 2787579659 bytes (2659 MiB)

Extracting archive: /content/gdrive/MyDrive/birdsFinal.7z
--
Path = /content/gdrive/MyDrive/birdsFinal.7z
Type = 7z
Physical Size = 2787579659
Headers Size = 3160162
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 1578           0% 1578 - test/ABBOTTS BABBLER/004.jpg                                          0% 1803 - test/AFRICAN EMERALD CUCKOO/149.jpg                                               

# Инициализация нужных в дальнейшем переменных

In [ ]:
train_dir = "/content/train"
test_dir = "/content/test"
val_dir = "/content/val"
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 16
# Experiment 3(100)
# nb_train_samples = 96416
# nb_validation_samples = 20458
# nb_test_samples = 21176

#Experiment 1(150)
nb_train_samples = 117910
nb_validation_samples = 25077
nb_test_samples = 25798

# Imports для СНС

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras.applications import VGG19
from keras.optimizers import Adam

# Инициализация СНС

In [ ]:
from keras.applications import MobileNetV2
base_model = MobileNetV2(include_top = False,
                       weights = 'imagenet',
                       input_shape = input_shape)



9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
base_model.summary()


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(525, activation='softmax'))

In [ ]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_2 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=7e-5),
              metrics=['accuracy'])

# Data Generator (Определение классов из наших директорий)

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)


In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size
    )

Found 117802 images belonging to 525 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size
    )


Found 25054 images belonging to 525 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size
    )

Found 25774 images belonging to 525 classes.


# Обучение

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [ ]:
model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[es])

Epoch 1/20
7363/7363 [==============================] - 367s 49ms/step - loss: 4.0441 - accuracy: 0.2333 - val_loss: 1.9211 - val_accuracy: 0.5713
Epoch 2/20
7363/7363 [==============================] - 367s 50ms/step - loss: 1.8619 - accuracy: 0.5504 - val_loss: 1.1930 - val_accuracy: 0.7033
Epoch 3/20
7363/7363 [==============================] - 389s 53ms/step - loss: 1.4055 - accuracy: 0.6400 - val_loss: 0.9762 - val_accuracy: 0.7492
Epoch 4/20
7363/7363 [==============================] - 365s 50ms/step - loss: 1.2018 - accuracy: 0.6850 - val_loss: 0.8693 - val_accuracy: 0.7756
Epoch 5/20
7363/7363 [==============================] - 352s 48ms/step - loss: 1.0767 - accuracy: 0.7146 - val_loss: 0.8051 - val_accuracy: 0.7898
Epoch 6/20
7363/7363 [==============================] - 353s 48ms/step - loss: 0.9852 - accuracy: 0.7349 - val_loss: 0.7568 - val_accuracy: 0.8022
Epoch 7/20
7363/7363 [==============================] - 351s 48ms/step - loss: 0.9115 - accuracy: 0.7531 - val_loss: 0

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


<ipython-input-38-95bfb2ba4b2e>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


In [ ]:
print("accuracy на тестовых данных: %.2f%%" % (scores[1]*100))


accuracy на тестовых данных: 86.39%


In [ ]:
model.save(filepath="/content/gdrive/MyDrive/modelNewData.hdf5")

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("/content/modelBase8527.hdf5")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

In [ ]:
import keras

In [ ]:
flag = False
for layer in model.layers:
    if "block_16_expand" in layer.name:
      layer.trainable = True
      flag = True
    if isinstance(layer, keras.layers.BatchNormalization):
      layer.trainable = False
      continue
    if flag:
      layer.trainable = True

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

In [ ]:
model

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[es])

Epoch 1/20
1785/1785 [==============================] - 413s 229ms/step - loss: 0.3559 - accuracy: 0.8921 - val_loss: 0.5261 - val_accuracy: 0.8635
Epoch 2/20
1785/1785 [==============================] - 400s 224ms/step - loss: 0.3189 - accuracy: 0.9030 - val_loss: 0.5241 - val_accuracy: 0.8649
Epoch 3/20
1505/1785 [========================>.....] - ETA: 48s - loss: 0.2960 - accuracy: 0.9099

KeyboardInterrupt: ignored